# Fraud Detection Solution Training Notebook with My Own Docker

This notebook shows the 1st time training process of a fraud detection model using graph neural networks in the Solution.

This notebook assumes the transaction data has been dumped out from the Neptune DB, and copied to S3 bukets. So the input data has already in S3.

Then we create a launch of training job using the SageMaker framework estimator to train a graph neural network model with DGL.

### Major difference from the Solution Training Notebook is using my own docker image

In [1]:
! pip uninstall sagemaker -y
! pip install sagemaker

Found existing installation: sagemaker 2.23.6
Uninstalling sagemaker-2.23.6:
  Successfully uninstalled sagemaker-2.23.6
     |████████████████████████████████| 399 kB 1.3 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.24.0-py2.py3-none-any.whl size=560477 sha256=1ff4d1f86ee80c28a83a66aa2a6d549759f6e12b655f1d131e262381bd9340d6
  Stored in directory: /home/ec2-user/.cache/pip/wheels/37/51/a5/be3cd84d4ee4348854202a9379aa2fa8509c1a87b7d1686561
Successfully built sagemaker
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import sagemaker
from sagemaker_graph_fraud_detection import config, container_build

role = config.role
sess = sagemaker.Session()

## Define Data Location

### Loading Pre-processed data from S3

The dataset used in this Solution is the [IEEE-CIS Fraud Detection dataset](https://www.kaggle.com/c/ieee-fraud-detection/data?select=train_transaction.csv) which is a typical example of financial transactions dataset that many companies have. The dataset consists of two tables:

* **Transactions**: Records transactions and metadata about transactions between two users. Examples of columns include the product code for the transaction and features on the card used for the transaction. 
* **Identity**: Contains information about the identity users performing transactions. Examples of columns here include the device type and device ids used.

This notebook assumes that the two data tables had been pre-processed, mimicing the 1st time data preparation. 

**Current version uses the pre-processed data in nearly raw format, include all relation files, a feature file, a tag file, and a test index files.**

In [3]:
# Replace with an S3 location or local path to point to customers' own dataset
data_location = 'fraud-detection-solution'
raw_data_folder = 'raw_data'
processed_data_folder = 'processed_data'
model_output_folder = 'model_output'

processed_data = 's3://{}/{}'.format(data_location, processed_data_folder)
output_path = 's3://{}/{}'.format(data_location, model_output_folder)

print(processed_data)
print(output_path)

s3://fraud-detection-solution/processed_data
s3://fraud-detection-solution/model_output


### View the pre-processed data

Here come the pre-processed file lists

## Need to modified this to Neptune dump files!

In [4]:
from os import path
from sagemaker.s3 import S3Downloader
processed_files = S3Downloader.list(processed_data)


## Train Graph Neural Network with DGL

Graph Neural Networks work by learning representation for nodes or edges of a graph that are well suited for some downstream task. We can model the fraud detection problem as a node classification task, and the goal of the graph neural network would be to learn how to use information from the topology of the sub-graph for each transaction node to transform the node's features to a representation space where the node can be easily classified as fraud or not.

Specifically, we will be using a relational graph convolutional neural network model (R-GCN) on a heterogeneous graph since we have nodes and edges of different types.

### Hyperparameters

To train the graph neural network, we need to define a few hyperparameters that determine properties such as the class of graph neural network models we will be using, the network architecture and the optimizer and optimization parameters. 

Here we're setting only a few of the hyperparameters, to see all the hyperparameters and their default values, see `dgl-fraud-detection/estimator_fns.py`. The parameters set below are:

* **`nodes`** is the name of the file that contains the `node_id`s of the target nodes and the node features.
* **`edges`** is a regular expression that when expanded lists all the filenames for the edgelists
* **`labels`** is the name of the file tha contains the target `node_id`s and their labels

The following hyperparameters can be tuned and adjusted to improve model performance
* **embedding-size** is the size of the embedding dimension for non target nodes
* **n-layers** is the number of GNN layers in the model
* **n-epochs** is the number of training epochs for the model training job
* **optimizer** is the optimization algorithm used for gradient based parameter updates
* **lr** is the learning rate for parameter updates


In [5]:
edges = ",".join(map(lambda x: x.split("/")[-1], [file for file in processed_files if "relation" in file]))
params = {'nodes' : 'features.csv',
          'edges': 'relation*',
          'labels': 'tags.csv',
          'embedding-size': 64,
          'n-layers': 2,
          'n-epochs': 10,
          'optimizer': 'adam',
          'lr': 1e-2
        }

# print("Graph will be constructed using the following edgelists:\n{}" .format('\n'.join(edges.split(","))))

### Create and Fit SageMaker Pytorch Estimator

With the hyperparameters defined, then kick off the training job. Here use the Deep Graph Library (DGL), with Pytorch as the backend deep learning framework, to define and train the graph neural network. Amazon SageMaker makes it do this with the Framework estimators which have the deep learning frameworks already setup. Here, we create a SageMaker Pytorch estimator and pass in our model training script, hyperparameters, as well as the number and type of training instances specified.

Then `fit` the estimator on the the training data location in S3.

In [7]:
from time import strftime, gmtime
from sagemaker.estimator import Estimator

estimator = Estimator(image_uri='510768346845.dkr.ecr.cn-north-1.amazonaws.com.cn/pytorch-extending-our-containers-gnn-fraud-detection-solution:latest',
                      role=role,
                      train_instance_count=1,
                      train_instance_type='ml.c5.4xlarge',
                      hyperparameters=params,
                      output_path=output_path,
                      disable_profiler=True,
                      sagemaker_session=sess)

training_job_name = "{}-{}".format('GNN_FD_SL_DGL_Train', strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
print(training_job_name)

estimator.fit({'train': processed_data}, job_name=training_job_name)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


JamesTrain01-2021-01-24-06-39-48
2021-01-24 06:39:48 Starting - Starting the training job...
2021-01-24 06:39:52 Starting - Launching requested ML instances......
2021-01-24 06:41:01 Starting - Preparing the instances for training......
2021-01-24 06:42:04 Downloading - Downloading input data...
2021-01-24 06:42:26 Training - Downloading the training image.........
2021-01-24 06:44:18 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-24 06:44:19,603 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-01-24 06:44:19,604 sagemaker-containers INFO     Failed to parse hyperparameter nodes value features.csv to Json.
Returning the value itself
2021-01-24 06:44:19,604 sagemaker-containers INFO     Failed to parse hyperparameter optimizer value adam to Json.
Returning the value itself
2021-01-24 06:44:19,604 sagemaker-

Read edges for relation6 from edgelist: /opt/ml/input/data/train/relation_card3_edgelist.csv
Read edges for relation7 from edgelist: /opt/ml/input/data/train/relation_id_34_edgelist.csv
Read edges for relation8 from edgelist: /opt/ml/input/data/train/relation_id_25_edgelist.csv
Read edges for relation9 from edgelist: /opt/ml/input/data/train/relation_id_29_edgelist.csv
Read edges for relation10 from edgelist: /opt/ml/input/data/train/relation_id_37_edgelist.csv
Read edges for relation11 from edgelist: /opt/ml/input/data/train/relation_card6_edgelist.csv
Read edges for relation12 from edgelist: /opt/ml/input/data/train/relation_id_13_edgelist.csv
Read edges for relation13 from edgelist: /opt/ml/input/data/train/relation_id_02_edgelist.csv
Read edges for relation14 from edgelist: /opt/ml/input/data/train/relation_id_27_edgelist.csv
Read edges for relation15 from edgelist: /opt/ml/input/data/train/relation_id_21_edgelist.csv
Read edges for relation16 from edgelist: /opt/ml/input/data/trai

Epoch 00000 | Time(s) 35.8723 | Loss 0.6157 | f1 0.0000 
Epoch 00001 | Time(s) 34.9996 | Loss 1.0748 | f1 0.0001 
Epoch 00002 | Time(s) 34.6909 | Loss 1.0398 | f1 0.0115 
Epoch 00003 | Time(s) 34.4711 | Loss 0.8684 | f1 0.0293 
Epoch 00004 | Time(s) 34.3011 | Loss 0.6471 | f1 0.0329 
Epoch 00005 | Time(s) 34.2298 | Loss 0.4132 | f1 0.1737 
Epoch 00006 | Time(s) 34.1801 | Loss 0.7476 | f1 0.1663 
Epoch 00007 | Time(s) 34.1623 | Loss 0.2851 | f1 0.0317 
Epoch 00008 | Time(s) 34.1129 | Loss 0.2750 | f1 0.0280 
Epoch 00009 | Time(s) 34.0983 | Loss 0.2999 | f1 0.0161 
Metrics
Confusion Matrix:
                                                    labels positive  labels negative
predicted positive              170             1692
predicted negative            17667           452903
                                f1: 0.0173, precision: 0.0913, recall: 0.0095, acc: 0.9590, roc: 0.4112, pr: 0.0436, ap: 0.0437
                             
Finished Model training
Saving model
2021-01-24 06:57:5

Once the training is completed, the training instances are automatically saved and SageMaker stores the trained model and evaluation results to a location in S3.